<a href="https://colab.research.google.com/github/MrAkimi/hello-world/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
!pip install keras request idx2numpy

  Stored in directory: /root/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /root/.cache/pip/wheels/7a/b5/69/3e0757b3086607e95db70661798fdf98a77a0bb79c54e1f320
  Stored in directory: /root/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /root/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /root/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /root/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request idx2numpy get post query-string public


In [3]:
import keras
import numpy as np

Using TensorFlow backend.


# Define the models

In [4]:
mobile_net = keras.applications.mobilenet.MobileNet(input_shape=(32, 32, 1), alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights=None, input_tensor=None, pooling=None, classes=10) # set inlude top if you dont want to use the default input size of (224, 224, 3)
vgg16 = keras.applications.vgg16.VGG16(include_top=True, weights=None, input_tensor=None, input_shape=(48, 48, 1), pooling=None, classes=10)
inception_v3 = keras.applications.inception_v3.InceptionV3(include_top=True, weights=None, input_tensor=None, input_shape=(139, 139,1), pooling=None, classes=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Get the Fashion MNIST dataset

In [0]:
import urllib.request
import gzip
import idx2numpy


def load_data():
    """Loads the Fashion MNIST dataset.
    # Arguments
        path: path where to cache the dataset locally
            (relative to ~/.keras/datasets).
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """

    path_x_train = urllib.request.urlretrieve('http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz')
    path_y_train = urllib.request.urlretrieve('http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz')
    path_x_test = urllib.request.urlretrieve('http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz')
    path_y_test = urllib.request.urlretrieve('http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

    with gzip.open('train-images-idx3-ubyte.gz', 'rb') as f:
        x_train = idx2numpy.convert_from_string(f.read())
    with gzip.open('train-labels-idx1-ubyte.gz', 'rb') as f:
        y_train = idx2numpy.convert_from_string(f.read())
    with gzip.open('t10k-images-idx3-ubyte.gz', 'rb') as f:
        x_test = idx2numpy.convert_from_string(f.read())
    with gzip.open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
        y_test  = idx2numpy.convert_from_string(f.read())

    return (x_train, y_train), (x_test, y_test)

In [6]:
batch_size = 128
num_classes = 10
epochs = 12

(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print('x_train shape:', x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (60000, 28, 28, 1)
x_train shape: (10000, 28, 28, 1)
y_train shape: (60000, 10)
y_test shape: (10000, 10)


# Train the Mobile Net

In [7]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train_padded = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), mode='constant')
x_test_padded = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), mode='constant')

print('x_train shape:', x_train_padded.shape)
print('x_test shape:', x_test_padded.shape)


mobile_net.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

mobile_net.fit(x_train_padded, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_padded, y_test))
score = mobile_net.evaluate(x_test_padded, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
x_train shape: (60000, 32, 32, 1)
x_test shape: (10000, 32, 32, 1)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 28s 474us/step - loss: 0.7202 - acc: 0.7401 - val_loss: 0.7222 - val_acc: 0.8022
Epoch 2/12
60000/60000 [==============================] - 17s 282us/step - loss: 0.4056 - acc: 0.8519 - val_loss: 0.5595 - val_acc: 0.8261
Epoch 3/12
60000/60000 [==============================] - 17s 281us/step - loss: 0.3395 - acc: 0.8748 - val_loss: 0.4327 - val_acc: 0.8483
Epoch 4/12
60000/60000 [==============================] - 17s 291us/step - loss: 0.2975 - acc: 0.8899 - val_loss: 0.3751 - val_acc: 0.8687
Epoch 5/12
60000/60000 [==============================] - 17s 290us/step - loss: 0.2650 - acc: 0.9018 - val_loss: 0.4755 - val_acc: 0.

# Train the Vgg16

In [0]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train_padded = np.pad(x_train, ((0,0), (10,10), (10,10), (0,0)), mode='constant')
x_test_padded = np.pad(x_test, ((0,0), (10,10), (10,10), (0,0)), mode='constant')

print('x_train shape:', x_train_padded.shape)
print('x_test shape:', x_test_padded.shape)


vgg16.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

vgg16.fit(x_train_padded, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_padded, y_test))
score = vgg16.evaluate(x_test_padded, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
x_train shape: (60000, 48, 48, 1)
x_test shape: (10000, 48, 48, 1)
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
45440/60000 [=====================>........] - ETA: 39s - loss: 2.3054 - acc: 0.1056

60000/60000 [==============================] - 167s 3ms/step - loss: 2.3048 - acc: 0.1045 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/12
51712/60000 [========================>.....] - ETA: 20s - loss: 2.3027 - acc: 0.1008

60000/60000 [==============================] - 154s 3ms/step - loss: 2.3027 - acc: 0.1002 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 3/12
54144/60000 [==========================>...] - ETA: 14s - loss: 1.9920 - acc: 0.1996

60000/60000 [==============================] - 154s 3ms/step - loss: 1.9068 - acc: 0.2308 - val_loss: 0.9976 - val_acc: 0.5602
Epoch 4/12
55040/60000 [==========================>...] - ETA: 12s - loss: 0.6622 - acc: 0.7420

60000/60000 [==============================] - 155s 3ms/step - loss: 0.6448 - acc: 0.7498 - val_loss: 0.4852 - val_acc: 0.8251
Epoch 5/12
55424/60000 [==========================>...] - ETA: 11s - loss: 0.3845 - acc: 0.8587

...

60000/60000 [==============================] - 154s 3ms/step - loss: 0.3106 - acc: 0.8863 - val_loss: 0.3124 - val_acc: 0.8866
Epoch 7/12
55552/60000 [==========================>...] - ETA: 10s - loss: 0.2662 - acc: 0.9031

60000/60000 [==============================] - 154s 3ms/step - loss: 0.2656 - acc: 0.9033 - val_loss: 0.3077 - val_acc: 0.8915
Epoch 8/12
55552/60000 [==========================>...] - ETA: 10s - loss: 0.2368 - acc: 0.9126

60000/60000 [==============================] - 154s 3ms/step - loss: 0.2364 - acc: 0.9128 - val_loss: 0.3039 - val_acc: 0.8960
Epoch 9/12
55552/60000 [==========================>...] - ETA: 10s - loss: 0.2109 - acc: 0.9234

60000/60000 [==============================] - 154s 3ms/step - loss: 0.2107 - acc: 0.9237 - val_loss: 0.2608 - val_acc: 0.9101
Epoch 10/12
55296/60000 [==========================>...] - ETA: 11s - loss: 0.1920 - acc: 0.9303

58880/60000 [============================>.] - ETA: 2s - loss: 0.1919 - acc: 0.9303